In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [2]:
trained_model_path = '/workdir/security/home/junjiehuang2468/paper/trained_models_weight/ember/'
best_trained_model = '2022-01-18 14:55/2w_epoch:0_test_acc:0.890858.pt'
data_path = "/workdir/security/home/junjiehuang2468/paper/data/ember2018/"
train_data_path = data_path + "malwares/" 
test_data_path = data_path + "test_malwares/" 

In [3]:
CUDA = True if torch.cuda.is_available() else False
NUM_WORKERS = 24
# BATCH_SIZE = 80
BATCH_SIZE = 10
LEAVE_BIT_NUMBER = 20000
KERNEL_SIZE = 500

In [4]:
LEAVE_BIT_NUMBER_DICT = {10000:'1w',20000:'2w'}

In [5]:
trainset = pd.read_csv(data_path + 'train_dataset.csv')
validset = pd.read_csv(data_path + 'valid_dataset.csv')
testset = pd.read_csv(data_path + 'test_dataset.csv')
testset = testset.iloc[np.argwhere(testset['labels'].values == 1).squeeze(),:]

In [6]:
class ExeDataset(Dataset):
    def __init__(self, malware_names, data_path, labels, leave_bit_num):
        self.malware_names = malware_names
        self.data_path = data_path
        self.labels = labels
        self.leave_bit_num = leave_bit_num

    def __len__(self):
        return len(self.malware_names)

    def __getitem__(self, idx):
        with open(self.data_path + self.malware_names[idx] + '.txt','rb') as fp:
            data = [bit+1 for bit in fp.read()[:self.leave_bit_num]]
            padding = [0]*(self.leave_bit_num-len(data))
            data = data + padding

        return np.array(data), np.array([self.labels[idx]])

In [7]:
train_dataset = ExeDataset(
    trainset["id"].tolist(), 
    train_data_path, 
    trainset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
valid_dataset = ExeDataset(
    validset["id"].tolist(), 
    train_data_path, 
    validset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)
test_dataset = ExeDataset(
    testset["id"].tolist(), 
    test_data_path, 
    testset["labels"].tolist(), 
    LEAVE_BIT_NUMBER
)

In [8]:
trainloader = DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
validloader = DataLoader(
    dataset = valid_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)
testloader = DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = False,
    num_workers = NUM_WORKERS,
    pin_memory = True
)

In [9]:
class Model(nn.Module):
    def __init__(self, data_length = 2e6, kernel_size = 500):
        super().__init__()
        self.embedding = nn.Embedding(257, 8, padding_idx=0)
        self.conv_layer_1 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        # self.bn_1 = nn.BatchNorm1d(128)
        self.conv_layer_2 = nn.Conv1d(4, 128, kernel_size, stride = kernel_size, bias = True)
        self.pool_layer_2 = nn.MaxPool1d(data_length//kernel_size)
        self.fc_layer_3 = nn.Linear(128, 128)
        self.fc_layer_4 = nn.Linear(128, 2)

    def forward(self, input_, loss_fn):
        one_hot_x = F.one_hot(input_,num_classes=257).float()
        one_hot_x.requires_grad = True
        one_hot_x.retain_grad()
        batch_acc = []
        batch_grad = []
        for _ in range(13):
            x = one_hot_x @ self.embedding.weight
            x = torch.transpose(x, -1, -2)

            x_conv_1 = self.conv_layer_1(x[:,:4,:])
            x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

            x = x_conv_1*x_conv_2
            del x_conv_1,x_conv_2
            x = self.pool_layer_2(x).squeeze()
            
            x = self.fc_layer_3(x)
            x = self.fc_layer_4(x)
            
            acc = torch.argmax(torch.softmax(x,dim=-1),dim=-1).float().mean()
            batch_acc.append(acc.tolist())
            
            mislead_labels = torch.cat((torch.ones(len(x)).reshape(-1,1),torch.zeros(len(x)).reshape(-1,1)),dim=1).cuda()
            loss = loss_fn(x,mislead_labels)
            loss.backward()
            
            padding_place_mask = (input_ == 0)
            shape = padding_place_mask.shape
            expand_padding_place_mask = padding_place_mask.unsqueeze(-1).expand(shape[0],shape[1],257)
            grad_min_idx = torch.argmin(one_hot_x.grad,dim=-1)
            grad_sez_mask = (one_hot_x.grad <= 0)
            grad_mask = (expand_padding_place_mask & grad_sez_mask)
            batch_grad.append(torch.div(
                input=(expand_padding_place_mask.float()*one_hot_x.grad).sum(),
                other=expand_padding_place_mask.float().sum()
            ).detach().cpu().numpy())
            
            ########## wrong but work ###########
            # one_hot_x.data = F.one_hot(torch.argmin(
            #     one_hot_x.grad*grad_mask,
            #     dim=-1
            # ),num_classes=257).float()
            ########## wrong but work ###########
            one_hot_x.data = F.one_hot(torch.argmin(
                one_hot_x.grad,
                dim=-1
            ),num_classes=257).float()*(expand_padding_place_mask.float()) + one_hot_x*(1 - expand_padding_place_mask.float())
#             new_x = torch.argmin(grad_mask.float()*one_hot_x.grad,dim=-1)
#             grad_all_gz_mask = grad_sez_mask.sum(dim=-1) == 257
#             mask = ((~grad_all_gz_mask) & padding_place_mask)
#             temp = (1 - mask.float())*input_ + mask.float()*new_x
#             one_hot_x.data = F.one_hot(temp.long(),num_classes=257).float()
            
            # one_hot_x.grad.zero_()
            
        x = one_hot_x @ self.embedding.weight
        x = torch.transpose(x, -1, -2)

        x_conv_1 = self.conv_layer_1(x[:,:4,:])
        x_conv_2 = torch.sigmoid(self.conv_layer_2(x[:,4:,:]))

        x = x_conv_1*x_conv_2
        del x_conv_1,x_conv_2
        x = self.pool_layer_2(x).squeeze()

        x = self.fc_layer_3(x)
        x = self.fc_layer_4(x)
        # return x.detach().cpu().numpy(),batch_acc,one_hot_x
        return x.detach().cpu().numpy(),batch_acc,batch_grad

In [10]:
model = Model(data_length=LEAVE_BIT_NUMBER,kernel_size=KERNEL_SIZE)

ce_loss = nn.CrossEntropyLoss()
optim = Adam(model.parameters())

model = model.cuda() if CUDA else model
ce_loss = ce_loss.cuda() if CUDA else ce_less

In [11]:
model.load_state_dict(torch.load(trained_model_path + best_trained_model))

<All keys matched successfully>

In [12]:
model.eval()
acc = []
preds = []
labels = []
total_batch_acc = []
total_batch_grad = []
bar = tqdm(testloader)
# bar = tqdm(validloader)
for step, (batch_data,batch_label) in enumerate(bar):
    batch_data = batch_data.cuda() if CUDA else batch_data
    batch_label = batch_label.cuda() if CUDA else batch_label
    batch_label = batch_label.squeeze()
    temp_label = torch.zeros((len(batch_label),2))
    for idx,target in enumerate(batch_label.squeeze()): temp_label[idx,target] = 1
    temp_label = temp_label.cuda() if CUDA else temp_label

#     pred,input_,one_hot_x = model(batch_data,ce_loss)
#     break
    pred,batch_acc,batch_grad = model(batch_data,ce_loss)
#     break
    pred = np.argmax(pred,1)
    batch_label = batch_label.cpu().data.numpy()
    
    preds.extend(pred.tolist())
    labels.extend(batch_label.tolist())
    
    temp_acc = (batch_label == pred).mean()
    acc.append(temp_acc)
    total_batch_acc.append(batch_acc + [temp_acc])
    total_batch_grad.append([0] + batch_grad)
    total_batch_acc_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_acc,axis=0))) + ']'
    total_batch_grad_str = '[' + ' '.join(map(lambda x: '%.10f'%x,np.mean(total_batch_grad,axis=0))) + ']'
    print(total_batch_acc_str)
    print(total_batch_grad_str)
    print(f"test：{temp_acc:4f}, test mean: {np.mean(acc):4f}")

  0%|          | 1/10000 [00:02<7:04:01,  2.54s/it]

[0.6999999881 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000]
[0.0000000000 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005 -0.0000004005]
test：0.000000, test mean: 0.000000


  0%|          | 2/10000 [00:02<3:18:48,  1.19s/it]

[0.6999999881 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000 0.0000000000]
[0.0000000000 -0.0000003768 -0.0000003768 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767 -0.0000003767]
test：0.000000, test mean: 0.000000


  0%|          | 3/10000 [00:03<2:05:53,  1.32it/s]

...

  6%|▌         | 568/10000 [02:14<37:25,  4.20it/s]

[0.8603873394 0.0216549299 0.0170774650 0.0156690143 0.0151408453 0.0147887326 0.0146126763 0.0146126763 0.0144366199 0.0142605636 0.0142605636 0.0142605636 0.0142605636 0.0142605634]
[0.0000000000 -0.0000002322 -0.0000002296 -0.0000002292 -0.0000002291 -0.0000002290 -0.0000002289 -0.0000002288 -0.0000002287 -0.0000002286 -0.0000002285 -0.0000002284 -0.0000002282 -0.0000002281]
test：0.000000, test mean: 0.014261


  6%|▌         | 569/10000 [02:15<37:18,  4.21it/s]

[0.8604569575 0.0216168720 0.0170474519 0.0156414765 0.0151142357 0.0147627419 0.0145869949 0.0145869949 0.0144112480 0.0142355011 0.0142355011 0.0142355011 0.0142355011 0.0142355009]
[0.0000000000 -0.0000002321 -0.0000002294 -0.0000002290 -0.0000002290 -0.0000002289 -0.0000002288 -0.0000002287 -0.0000002286 -0.0000002285 -0.0000002284 -0.0000002282 -0.0000002281 -0.0000002280]
test：0.000000, test mean: 0.014236


KeyboardInterrupt: 

In [ ]:
# import pickle

In [ ]:
# with open('/workdir/security/home/junjiehuang2468/paper/results/ember/my_way/cumulative_gradients_v2_13iter_grad.txt','wb') as fp:
#     pickle.dump(total_batch_grad,fp)